<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/stock_market_26_jul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.28 requests gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.97.1


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai')


In [3]:
import gradio as gr
import openai
import requests

# Set your OpenAI and stock market API keys from Alpha Vantage here

STOCK_API_KEY = '4CVR0EO3D90OX22H'



# Function to get stock symbol from company name using Alpha Vantage SYMBOL_SEARCH
def get_stock_symbol(company_name):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'SYMBOL_SEARCH',
        'keywords': company_name,
        'apikey': STOCK_API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    try:
        # Return the best match's symbol
        symbol = data['bestMatches'][0]['1. symbol']
        return symbol
    except (KeyError, IndexError):
        return None

# Function to get stock quote from Alpha Vantage
def get_stock_quote(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'GLOBAL_QUOTE',
        'symbol': symbol,
        'apikey': STOCK_API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    try:
        quote = data['Global Quote']
        price = quote['05. price']
        return f"The latest price for {symbol} is ${price}"
    except KeyError:
        return "Could not fetch the stock data. Please try again later."

# Function to interact with OpenAI GPT
def ask_gpt(prompt):
    try:
        # Interact with OpenAI GPT using gpt-3.5-turbo
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful stock market assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=250,
            temperature=0.2
        )
        return response['choices'][0]['message']['content'].strip()

    except openai.error.OpenAIError as e:
        return f"An error occurred while contacting OpenAI: {e}"

    except Exception as e:
        return f"An unexpected error occurred: {e}"

# Gradio interface function
def stock_info(company_name):
    symbol = get_stock_symbol(company_name)
    if not symbol:
        return f"Could not find stock symbol for {company_name}.", ""

    stock_price = get_stock_quote(symbol)
    gpt_response = ask_gpt(f"What do you think about {company_name} stock? Which all stock exchanges is the stock listed, is it a good long term investment , give some historical financial milestones and annoucements")

    return stock_price, gpt_response

# Create Gradio interface
with gr.Blocks(title="DDS Stock Market and GPT Assistant") as demo:
    gr.Markdown("<h1 style='text-align: center;'>Stock Market and ChatGPT Assistant</h1>")
    gr.Markdown("<p style='text-align: center;'>This app provides the latest stock prices and Long term insights from ChatGPT. Enter a company name below:</p>")
    company_name = gr.Textbox(label="Enter Company Name")
    stock_output = gr.Textbox(label="Stock Price")
    gpt_output = gr.Textbox(label="AI Opinion")

    submit_button = gr.Button("Get Stock Info")

    submit_button.click(fn=stock_info, inputs=company_name, outputs=[stock_output, gpt_output])

# Launch the Gradio app
demo.launch()

#65 line of code

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://605b236ac92ef4b072.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
